In [1]:
import librosa
import numpy as np
import pandas as pd
import os
import tensorflow as tf

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
emotions=['angry','disgust','fear','happy','neutral','sad','surprise']

In [4]:
main_path="C:\\Users\\navee\\NullClass-Tasks\\Emotion_Detection_voice\\audio\\train"

In [5]:
def load_extract_features(folder_path,emotion_class):
    features=[]
    labels=[]
    
    for file_name in os.listdir(folder_path):
        file_path=os.path.join(folder_path,file_name)
        audio_data,sampling_rate=librosa.load(file_path)
        
        mfccs=librosa.feature.mfcc(y=audio_data,sr=sampling_rate,n_mfcc=10)
        chroma=librosa.feature.chroma_stft(y=audio_data,sr=sampling_rate)
        mel=librosa.feature.melspectrogram(y=audio_data,sr=sampling_rate)
        
        mfccs_flat=np.mean(mfccs,axis=1)
        chroma_flat=np.mean(chroma,axis=1)
        mel_flat=np.mean(mel,axis=1)
        
        audio_features=np.concatenate([mfccs_flat,chroma_flat,mel_flat])
        
        features.append(audio_features)
        labels.append(emotion_class)
    
    return features,labels

In [6]:
all_features=[]
all_labels=[]

In [7]:
for emotion in emotions:
    emotion_folder=os.path.join(main_path,emotion)
    features,labels=load_extract_features(emotion_folder,emotion)
    all_features.append(features)
    all_labels.append(labels)

In [8]:
all_features=np.concatenate(all_features)

In [9]:
all_labels=np.concatenate(all_labels)

In [10]:
x_audio=np.array(all_features)
y_audio=np.array(all_labels)

In [11]:
x_audio.shape

(2496, 150)

In [12]:
y_audio.shape

(2496,)

In [13]:
for i in range(len(y_audio)):
    if y_audio[i]=='angry':
        y_audio[i]=0
    
    elif y_audio[i]=='disgust':
        y_audio[i]=1
    
    elif y_audio[i]=='fear':
        y_audio[i]=2
    
    elif y_audio[i]=='happy':
        y_audio[i]=3
    
    elif y_audio[i]=='neutral':
        y_audio[i]=4
    
    elif y_audio[i]=='sad':
        y_audio[i]=5
        
    elif y_audio[i]=='surprise':
        y_audio[i]=6

In [14]:
x_audio=x_audio.reshape((x_audio.shape[0],1,x_audio.shape[1]))

In [15]:
x_audio.shape

(2496, 1, 150)

In [16]:
y_audio=tf.keras.utils.to_categorical(y_audio,num_classes=7)

In [17]:
y_audio=np.argmax(y_audio,axis=1)

In [18]:
train_data=tf.keras.utils.image_dataset_from_directory("images\\train",
                                                      image_size=(48,48),
                                                      color_mode='grayscale',
                                                      label_mode='categorical',
                                                      batch_size=32,
                                                      shuffle=True)

Found 28709 files belonging to 7 classes.


In [19]:
x_image,y_image=[images for images,_ in train_data],[labels for _,labels in train_data]

In [20]:
x_image=np.concatenate(x_image)
y_image=np.concatenate(y_image)

In [21]:
y_image=np.argmax(y_image,axis=1)

In [22]:
angry=[]
disgust=[]
fear=[]
happy=[]
neutral=[]
sad=[]
surprise=[]

In [23]:
for i in range(len(x_image)):
    if y_image[i]==0 and len(angry)<384:
        angry.append(x_image[i])
        
    elif y_image[i]==1 and len(disgust)<384:
        disgust.append(x_image[i])
        
    elif y_image[i]==2 and len(fear)<384:
        fear.append(x_image[i])
        
    elif y_image[i]==3 and len(happy)<384:
        happy.append(x_image[i])
        
    elif y_image[i]==4 and len(neutral)<192:
        neutral.append(x_image[i])
        
    elif y_image[i]==5 and len(sad)<384:
        sad.append(x_image[i])
        
    elif y_image[i]==6 and len(surprise)<384:
        surprise.append(x_image[i])

In [24]:
x_image=angry+disgust+fear+happy+neutral+sad+surprise

In [25]:
x_image=np.array(x_image)

In [26]:
y_image=[]

In [27]:
y_image=[0]*384+[1]*384+[2]*384+[3]*384+[4]*192+[5]*384+[6]*384

In [28]:
count =0
for i in range(len(y_audio)):
    if y_audio[i]!=y_image[i]:
        count+=1

if count==0:
    print("both are same")
else:
    print("both are not same")

both are same


In [29]:
y=tf.keras.utils.to_categorical(y_image)

In [30]:
x_audio_tr,x_audio_te,x_image_tr,x_image_te,y_tr,y_te=train_test_split(x_audio,x_image,y,test_size=0.2,random_state=0)

In [31]:
y_te

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [32]:
audio_input=tf.keras.layers.Input(shape=(1,150),name='audio_input')

model_lstm=tf.keras.models.Sequential(name='model_lstm')

model_lstm.add(tf.keras.layers.LSTM(64,return_sequences=True))

model_lstm.add(tf.keras.layers.LSTM(128,return_sequences=True))

model_lstm.add(tf.keras.layers.LSTM(256))

model_lstm.add(tf.keras.layers.Dense(128, activation='relu'))
model_lstm.add(tf.keras.layers.Dense(64, activation='relu'))
model_lstm.add(tf.keras.layers.Dense(7, activation='softmax'))

audio_output=model_lstm(audio_input)

In [33]:
image_input=tf.keras.layers.Input(shape=(48,48,1),name='image_input')

model_cnn=tf.keras.models.Sequential()

model_cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.Dropout(0.2))
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model_cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model_cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model_cnn.add(tf.keras.layers.Flatten())

model_cnn.add(tf.keras.layers.Dense(128,activation='relu'))
model_cnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_cnn.add(tf.keras.layers.Dense(7,activation='softmax'))

image_output=model_cnn(image_input)

In [34]:
merged=tf.keras.layers.Concatenate()([audio_output,image_output])

In [35]:
final_output=tf.keras.layers.Dense(7,activation='softmax',name='final_output')(merged)

In [36]:
combined_model=tf.keras.models.Model(inputs=[audio_input,image_input],outputs=[final_output])

In [37]:
combined_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
combined_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 audio_input (InputLayer)    [(None, 1, 150)]             0         []                            
                                                                                                  
 image_input (InputLayer)    [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 model_lstm (Sequential)     (None, 7)                    589703    ['audio_input[0][0]']         
                                                                                                  
 sequential (Sequential)     (None, 7)                    903047    ['image_input[0][0]']         
                                                                                              

In [39]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('Model_Integrated_weights.h5',monitor='val_accuracy',mode='max',save_best_only=True)

In [40]:
combined_model.fit([x_audio_tr,x_image_tr], y_tr, epochs=300, batch_size=32, validation_data=([x_audio_te,x_image_te],y_te),callbacks=checkpoint)

Epoch 1/300


63/63 [==============================] - 14s 117ms/step - loss: 1.9402 - accuracy: 0.1668 - val_loss: 1.9228 - val_accuracy: 0.1980
Epoch 2/300


C:\Users\navee\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


63/63 [==============================] - 8s 128ms/step - loss: 1.8932 - accuracy: 0.2255 - val_loss: 1.8858 - val_accuracy: 0.2260
Epoch 3/300
63/63 [==============================] - 8s 134ms/step - loss: 1.8751 - accuracy: 0.2239 - val_loss: 1.8781 - val_accuracy: 0.2240
Epoch 4/300
63/63 [==============================] - 8s 131ms/step - loss: 1.8585 - accuracy: 0.2325 - val_loss: 1.8528 - val_accuracy: 0.2380
Epoch 5/300
63/63 [==============================] - 8s 127ms/step - loss: 1.8389 - accuracy: 0.2485 - val_loss: 1.8479 - val_accuracy: 0.2380
Epoch 6/300
63/63 [==============================] - 8s 130ms/step - loss: 1.8356 - accuracy: 0.2525 - val_loss: 1.8288 - val_accuracy: 0.2420
Epoch 7/300
63/63 [==============================] - 8s 126ms/step - loss: 1.8180 - accuracy: 0.2781 - val_loss: 1.8213 - val_accuracy: 0.2360
Epoch 8/300
63/63 [==============================] - 8s 131ms/step - loss: 1.8065 - accuracy: 0.2745 - val_loss: 1.8118 - val_accuracy: 0.2620
Epoch 9/300

Epoch 59/300
63/63 [==============================] - 11s 169ms/step - loss: 1.4756 - accuracy: 0.4133 - val_loss: 1.4892 - val_accuracy: 0.4240
Epoch 60/300
63/63 [==============================] - 10s 158ms/step - loss: 1.4557 - accuracy: 0.4344 - val_loss: 1.4987 - val_accuracy: 0.4160
Epoch 61/300
63/63 [==============================] - 9s 141ms/step - loss: 1.4600 - accuracy: 0.4294 - val_loss: 1.4946 - val_accuracy: 0.4120
Epoch 62/300
63/63 [==============================] - 9s 142ms/step - loss: 1.4461 - accuracy: 0.4389 - val_loss: 1.5600 - val_accuracy: 0.3860
Epoch 63/300
63/63 [==============================] - 9s 140ms/step - loss: 1.4631 - accuracy: 0.4238 - val_loss: 1.5254 - val_accuracy: 0.4200
Epoch 64/300
63/63 [==============================] - 9s 144ms/step - loss: 1.4405 - accuracy: 0.4289 - val_loss: 1.4993 - val_accuracy: 0.4260
Epoch 65/300
63/63 [==============================] - 9s 137ms/step - loss: 1.4390 - accuracy: 0.4399 - val_loss: 1.4903 - val_accurac

Epoch 116/300
63/63 [==============================] - 12s 191ms/step - loss: 1.3270 - accuracy: 0.4734 - val_loss: 1.4230 - val_accuracy: 0.4540
Epoch 117/300
63/63 [==============================] - 11s 170ms/step - loss: 1.3048 - accuracy: 0.4935 - val_loss: 1.4428 - val_accuracy: 0.4260
Epoch 118/300
63/63 [==============================] - 10s 155ms/step - loss: 1.3169 - accuracy: 0.4865 - val_loss: 1.4253 - val_accuracy: 0.4360
Epoch 119/300
63/63 [==============================] - 10s 158ms/step - loss: 1.2936 - accuracy: 0.4890 - val_loss: 1.4096 - val_accuracy: 0.4480
Epoch 120/300
63/63 [==============================] - 10s 156ms/step - loss: 1.2695 - accuracy: 0.4945 - val_loss: 1.4044 - val_accuracy: 0.4600
Epoch 121/300
63/63 [==============================] - 10s 155ms/step - loss: 1.2831 - accuracy: 0.4925 - val_loss: 1.4789 - val_accuracy: 0.4280
Epoch 122/300
63/63 [==============================] - 10s 159ms/step - loss: 1.2909 - accuracy: 0.4910 - val_loss: 1.4552 -

63/63 [==============================] - 10s 155ms/step - loss: 1.1438 - accuracy: 0.5726 - val_loss: 1.3290 - val_accuracy: 0.4940
Epoch 229/300
63/63 [==============================] - 10s 155ms/step - loss: 1.0408 - accuracy: 0.6207 - val_loss: 1.3485 - val_accuracy: 0.5280
Epoch 230/300
63/63 [==============================] - 10s 155ms/step - loss: 1.0682 - accuracy: 0.6087 - val_loss: 1.3028 - val_accuracy: 0.5540
Epoch 231/300
63/63 [==============================] - 10s 154ms/step - loss: 1.0247 - accuracy: 0.6263 - val_loss: 1.3544 - val_accuracy: 0.5500
Epoch 232/300
63/63 [==============================] - 10s 155ms/step - loss: 1.1582 - accuracy: 0.5661 - val_loss: 1.3378 - val_accuracy: 0.5180
Epoch 233/300
63/63 [==============================] - 10s 155ms/step - loss: 1.1002 - accuracy: 0.5852 - val_loss: 1.3344 - val_accuracy: 0.5280
Epoch 234/300
63/63 [==============================] - 10s 155ms/step - loss: 1.1026 - accuracy: 0.5902 - val_loss: 1.3579 - val_accuracy:

In [41]:
combined_model.load_weights('Model_Integrated_weights.h5')

In [42]:
combined_model.evaluate([x_audio_te,x_image_te],y_te)

16/16 [==============================] - 1s 35ms/step - loss: 1.1745 - accuracy: 0.6060


[1.1744788885116577, 0.6060000061988831]

In [43]:
model_json=combined_model.to_json()
with open("model_integrated_a.json","w") as json_file:
    json_file.write(model_json)